In [4]:
import pickle
import numpy as np

In [5]:
#adir = "/home/nanci/ray_results/MultiPPO_2021-10-11_10-54-46/MultiPPO_coverage_2d1d6_00000/gaussian/eval_adv-2d1d6-007500_dataset.pkl"
adir = "/home/nanci/ray_results/MultiPPO_2022-04-12_11-07-33/MultiPPO_coverage_4799f_00000/gaussian/eval_adv-4799f-015000_dataset_with_label.pkl"



dataset = pickle.load(open(adir,'rb'))
dataset = dataset[dataset.agent > 0]
dataset = np.array([d[0] for d in dataset.dataset])

In [6]:
dataset.shape[0]/5

34500.0

In [7]:
dataset.shape

(172500, 128)

In [57]:
#adir2 = "/home/nanci/ray_results/MultiPPO_2021-10-11_10-54-46/MultiPPO_coverage_2d1d6_00000/gaussian/dataset_adv.pkl"

adir2 = "/home/nanci/ray_results/MultiPPO_2022-04-12_11-07-33/MultiPPO_coverage_4799f_00000/gaussian/eval_adv-4799f-015000_dataset_adv_with_label.pkl"

dataset2 = pickle.load(open(adir2,'rb'))
dataset2 = dataset2[dataset2.agent == 0]
dataset_adv = np.array([d[0] for d in dataset2.dataset]) #np.array(dataset2['dataset'])
dataset_adv = dataset_adv.reshape(-1,128)

In [58]:
dataset_adv[0].shape

(128,)

In [71]:
del VAE

In [8]:
import torch
import numpy as np
#from vae_net2 import VAE
import argparse
from tqdm import tqdm
import time
import scipy.io
import random
import math

from utils.test import test_function

In [9]:
import torch
import torch.nn as nn


# VAE with one stochastic layer z
class VAE(nn.Module):

    def __init__(self, args, d, h_num, scaled=True):
        super(VAE, self).__init__()
        self.dim = 128
        self.Nz = 10
        self.hid_num = h_num
        self.output_type = 'gaussian'
        self.decoder_type = 'gaussian'
        self.scaled_mean = scaled
        self.fc1 = nn.Linear(d, h_num)
        self.fc2_mu = nn.Linear(h_num, self.Nz)
        self.fc2_sigma = nn.Linear(h_num, self.Nz)
        self.fc3 = nn.Linear(self.Nz, h_num)
        if self.decoder_type == 'gaussian':
            self.fc4_mu = nn.Linear(h_num, d)
            self.fc4_sigma = nn.Linear(h_num, d)
        else:
            self.fc4 = nn.Linear(h_num, d)

    def forward(self, x):
        x = x.view(-1, self.dim)
        x = torch.tanh(self.fc1(x))
        mu_z = self.fc2_mu(x)
        log_sigma_z = self.fc2_sigma(x)
        eps = torch.randn_like(mu_z)
        x = mu_z + torch.exp(log_sigma_z) * eps
        x = torch.tanh(self.fc3(x))
        if self.output_type == 'gaussian':
            if self.scaled_mean:
                mu = torch.sigmoid(self.fc4_mu(x))
            else:
                mu = self.fc4_mu(x)
            log_sigma = self.fc4_sigma(x)
            return mu, mu_z, log_sigma, log_sigma_z
        else:
            x = self.fc4(x)
            return x, mu_z, '_', log_sigma_z
        



In [19]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                           shuffle=True, num_workers=2)
# define the model
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [20]:
epochs = 3
decoder_type = 'gaussian'

batchsize = 256

In [21]:
net = VAE( args = None, d=128, h_num=20)
net.to(device)
optimizer = torch.optim.Adagrad(net.parameters())

In [22]:
for i in range(len(dataset)):
    if np.isnan(dataset[i]).any():
        print(dataset[i])

In [23]:
# train the model
start = time.time()
for epoch in range(epochs):
    # test
    if epoch % 10 == 1:
        #print(dataset[:-100].shape)
        test_elbo = test_function(net, batchsize, dataset=dataset[:-100], out_type='gaussian',
                                  testset=dataset[-100:], device=device)
        print('test average ELBO=', test_elbo)

    # iterations
    running_loss = 0.0
    runcnt = 0
    with tqdm(total=len(train_loader.dataset)) as progress_bar:
        for i, data in enumerate(train_loader, 0):
            train = data.to(device)
            

            optimizer.zero_grad()

            output = net(train.float())

            # the negative KL term
            negative_KL = (torch.ones_like(output[1]) + 2 * output[3] - output[1] * output[1] - torch.exp(
                2 * output[3])).sum(1) / 2

            # the log conditional prob term
            if decoder_type == 'gaussian':
                train_minus_mu = train - output[0]

                log_p_x_given_z = -torch.ones_like(train).sum(1) * np.log(2 * math.pi) / 2 \
                                  - output[2].sum(1) / 2 - (
                                          train_minus_mu * train_minus_mu / (2 * torch.exp(output[2]))).sum(1)
      
            else:
                log_p_x_given_z = torch.sum(output[0] * train - torch.log(1 + torch.exp(output[0])), 1)
                
                
            #weights explode so scale it down?
            log_p_x_given_z /= data.shape[0] * 128/64

            # update parameters
            loss = -negative_KL.mean() - log_p_x_given_z.mean()
            loss.backward()
            optimizer.step()
            running_loss -= negative_KL.sum().item()
            running_loss -= log_p_x_given_z.sum().item()

            # progress bar
            progress_bar.set_postfix(loss=loss.mean().item())
            progress_bar.update(data.size(0))
            runcnt+=1

    print('[%d] loss: %.3f' % (epoch + 1, running_loss / runcnt))

print('Finished Training, time cost', time.time() - start)

PATH = 'save/vae_' + '_dataset_'  + '_decoder_' + 'readapt_coop.pth'
torch.save(net.state_dict(), PATH)


  0%|          | 0/172500 [00:00<?, ?it/s]

[1] loss: -47.204
test average ELBO= 187.78648376464844


  0%|          | 0/172500 [00:00<?, ?it/s]

[2] loss: -325.140


100%|██████████| 172500/172500 [02:21<00:00, 1220.11it/s, loss=-6.93]

[3] loss: -503.415
Finished Training, time cost 411.60105299949646


In [25]:
dataset[0]

array([ 0.        ,  1.9554069 ,  2.5773318 ,  0.        ,  6.871078  ,
       11.239436  ,  3.9592528 ,  4.0226893 ,  0.        ,  0.        ,
        2.7727277 ,  0.        ,  0.        ,  0.        ,  4.02565   ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.8181938 ,
        3.8436553 ,  0.        ,  0.        ,  4.386832  ,  0.        ,
        5.0052257 ,  0.27447602,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [27]:
mu, mu_z, log_sigma, log_sigma_z = net.forward(torch.Tensor(dataset[0]))

In [28]:
mu

tensor([[9.9239e-01, 9.9826e-01, 2.9605e-01, 6.1260e-01, 9.9433e-01, 9.9565e-01,
         9.9570e-01, 9.9426e-01, 9.9809e-01, 9.9614e-01, 9.9490e-01, 9.4750e-01,
         9.9307e-01, 1.3457e-01, 9.7747e-01, 9.9242e-01, 9.9795e-01, 9.9349e-01,
         9.9296e-01, 5.2316e-01, 9.8977e-01, 9.9513e-01, 2.8743e-01, 7.9217e-01,
         9.9068e-01, 9.8050e-01, 9.9605e-01, 3.7802e-01, 9.8746e-01, 2.9398e-02,
         9.9301e-01, 4.1219e-01, 8.6880e-09, 7.9743e-09, 6.8640e-09, 7.1773e-09,
         8.5179e-09, 7.5585e-09, 4.2870e-09, 6.2457e-09, 1.0953e-08, 6.7820e-09,
         6.3545e-09, 6.8628e-09, 1.0098e-08, 6.8005e-09, 6.9060e-09, 1.2164e-08,
         8.1040e-09, 6.7816e-09, 6.9111e-09, 1.1230e-08, 6.9401e-09, 8.0183e-09,
         1.0905e-08, 4.9857e-09, 6.0034e-09, 7.4084e-09, 6.6933e-09, 1.0392e-08,
         7.9350e-09, 5.3685e-09, 8.2102e-09, 5.9740e-09, 1.2416e-08, 6.6210e-09,
         9.2343e-09, 7.4735e-09, 1.1644e-08, 9.1943e-09, 8.8573e-09, 7.1090e-09,
         7.2539e-09, 8.7484e